In [1]:
import pandas as pd
from itertools import combinations

In [2]:
nvalues = pd.read_csv('../data/nutritional_values.csv')
recipes = pd.read_csv('../data/filtered_recipes_100.csv',sep='|')

In [3]:
df = pd.merge(recipes, nvalues, on='Recipe_id', how='right')

In [4]:
filtros = ['Cena','Entrante','Plato principal','Acompañamiento']
df = df[df['Tipo'].isin(filtros)]

In [5]:
wanted_cols =['Recipe_id','Categoria','energia','grasa','proteina','carbohidratos']
df = df[wanted_cols]
cc = list(combinations(df.Recipe_id,2)) #antes de esto hacer filtradosde comida y cena etc
df.set_index('Recipe_id',inplace=True)
df

,Categoria,energia,grasa,proteina,carbohidratos
Recipe_id,,,,,
74154,Recetas de Aperitivos y tapas,2761.935834,157.494375,194.583745,139.315964
74146,Recetas de Aperitivos y tapas,7069.600000,640.376000,144.868000,194.390900
74056,Recetas de Aperitivos y tapas,2355.060000,214.041700,19.874375,91.354000
74043,Recetas de Aperitivos y tapas,55.960000,2.923750,5.533000,1.835510
74020,Recetas de Aperitivos y tapas,714.260000,20.732000,17.247000,112.550000
...,...,...,...,...,...
9584,Recetas de Verduras,115.271250,8.252150,1.469375,8.835000
9110,Recetas de Verduras,899.006316,50.345149,47.644113,62.896002
9114,Recetas de Verduras,239.403614,10.907224,8.645025,23.228600


In [6]:
cc_cols = ['Recipe1','Recipe2'] #poner comida/cena whatever

combined_df = pd.DataFrame(columns = cc_cols)
combined_df['Recipe1'] = [c[0] for c in cc]
combined_df['Recipe2'] = [c[1] for c in cc]
combined_df

,Recipe1,Recipe2
0,74154,74146
1,74154,74056
2,74154,74043
3,74154,74020
4,74154,73919
...,...,...
4441685,9110,6618
4441686,9110,31007
4441687,9114,6618
4441688,9114,31007


In [7]:
macros = ['energia','grasa','proteina','carbohidratos']
for macro in macros:
    combined_df[macro] = [df[macro][c[1]]+df[macro][c[0]] for c in cc]
combined_df

,Recipe1,Recipe2,energia,grasa,proteina,carbohidratos
0,74154,74146,9831.535834,797.870375,339.451745,333.706864
1,74154,74056,5116.995834,371.536075,214.458120,230.669964
2,74154,74043,2817.895834,160.418125,200.116745,141.151474
3,74154,74020,3476.195834,178.226375,211.830745,251.865964
4,74154,73919,3812.655834,258.666175,206.817995,168.871964
...,...,...,...,...,...,...
4441685,9110,6618,1344.883816,75.276899,52.117863,113.546002
4441686,9110,31007,2425.652279,188.093066,93.760783,90.789029
4441687,9114,6618,685.281114,35.838974,13.118775,73.878600
4441688,9114,31007,1766.049577,148.655141,54.761695,51.121627


In [8]:
combined_df.to_csv('../data/combined.csv', index=False)

In [9]:
goals = { #por ejemplo, estos valores sería lo que falta después de fijar desayunos etc
    'goal_energia': 2000,
    'goal_grasa': 10,
    'goal_proteina': 20,
    'goal_carbohidratos': 30
}

#func para extraer los errores
for macro in macros:
    macro_error = macro + '_error'
    macro_goal = 'goal_' + macro
    combined_df[macro_error] = abs(goals[macro_goal]-combined_df[macro])

# error total
macro_errors = [macro+'_error' for macro in macros]
combined_df['total_error'] = combined_df[macro_errors].sum(axis=1) #los sumo pero podría ser otra operación 

combined_df

,Recipe1,Recipe2,energia,grasa,proteina,carbohidratos,energia_error,grasa_error,proteina_error,carbohidratos_error,total_error
0,1,5,138.342526,5.674834,1.590017,12.895666,1861.657474,4.325166,18.409983,17.104334,1901.496957
1,1,8,80.027500,0.425438,2.494250,10.141375,1919.972500,9.574562,17.505750,19.858625,1966.911437
2,1,10,636.090000,3.222000,30.738000,112.532000,1363.910000,6.778000,10.738000,82.532000,1463.958000
3,1,27,1693.297052,153.767161,12.876162,61.445363,306.702948,143.767161,7.123838,31.445363,489.039310
4,5,8,148.190026,5.996272,3.708267,18.573041,1851.809974,4.003728,16.291733,11.426959,1883.532394
5,5,10,704.252526,8.792834,31.952017,120.963666,1295.747474,1.207166,11.952017,90.963666,1399.870323
6,5,27,1761.459578,159.337995,14.090179,69.877029,238.540422,149.337995,5.909821,39.877029,433.665267
7,8,10,645.937500,3.543438,32.856250,118.209375,1354.062500,6.456562,12.856250,88.209375,1461.584687
8,8,27,1703.144552,154.088599,14.994412,67.122738,296.855448,144.088599,5.005588,37.122738,483.072373
9,10,27,2259.207052,156.885161,43.238162,169.513363,259.207052,146.885161,23.238162,139.513363,568.843738


In [17]:
def getClosestSum(n, df):
    return df.nsmallest(n, 'total_error')

In [18]:
getClosestSum(1, combined_df)

,Recipe1,Recipe2,energia,grasa,proteina,carbohidratos,energia_error,grasa_error,proteina_error,carbohidratos_error,total_error
6,5,27,1761.459578,159.337995,14.090179,69.877029,238.540422,149.337995,5.909821,39.877029,433.665267


In [19]:
getClosestSum(3, combined_df) #a partir de aquí extraer los ids, meterlos en una lista para pasarlos todos

,Recipe1,Recipe2,energia,grasa,proteina,carbohidratos,energia_error,grasa_error,proteina_error,carbohidratos_error,total_error
6,5,27,1761.459578,159.337995,14.090179,69.877029,238.540422,149.337995,5.909821,39.877029,433.665267
8,8,27,1703.144552,154.088599,14.994412,67.122738,296.855448,144.088599,5.005588,37.122738,483.072373
3,1,27,1693.297052,153.767161,12.876162,61.445363,306.702948,143.767161,7.123838,31.445363,489.039310
